### Libraries

In [2]:
import os
import numpy as np
import pandas as pd
from nitime.timeseries import TimeSeries
from nitime.analysis import SpectralAnalyzer, FilterAnalyzer, NormalizationAnalyzer
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

In [3]:
def lorentzian_function(x, s0, corner):
    return (s0*corner**2) / (x**2 + corner**2)

In [4]:
def multi_fractal_function(x, beta_low, beta_high, A, B, corner):
    return np.where(x < corner, A * x**beta_low, B * x**beta_high)

In [5]:
# Function for Gaussian smoothing with FWHM input
def gaussian_smoothing_with_fwhm(data, fwhm):
    """
    Apply Gaussian kernel smoothing to a time series with FWHM.

    Parameters:
    - data (numpy array): Input time-series data.
    - fwhm (float): Full-width at half-maximum (FWHM) for the Gaussian kernel.

    Returns:
    - smoothed_data (numpy array): Smoothed time-series data.
    """
    sigma = fwhm / 2.3548  # Convert FWHM to sigma
    smoothed_data = gaussian_filter1d(data, sigma=sigma)
    return smoothed_data

### Calculate knee frequencies for each site

In [21]:
# Define the base path where the .tsv files are stored
base_path = '/pscratch/sd/p/pakmasha/MBBN_data'

# Create a dictionary to store knee values for each site
f1_dict = {}
f2_dict = {}
seq_len_dict = {}
error_count = 0

# Walk through all subdirectories and process .npy files
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.npy'):
            # Construct full path to the .tsv file
            npy_file_path = os.path.join(root, file)
            # print(f"npy_file_path: {npy_file_path}")

            # Calculate the repetition time (TR) depending on the site
            site = file.split('_')[-2]
            if 'Amsterdam-AMC' in site:
                TR = 2.375
            elif 'Amsterdam-VUmc' in site:
                TR = 1.8
            elif 'Barcelona-HCPB' in site:
                TR = 2
            elif 'Bergen' in site:
                TR = 1.8
            elif 'Braga-UMinho-Braga-1.5T' in site:
                TR = 2
            elif 'Braga-UMinho-Braga-1.5T-act' in site:
                TR = 2
            elif 'Braga-UMinho-Braga-3T' in site:
                TR = 1
            elif 'Brazil' in site:
                TR = 2
            elif 'Cape-Town-UCT-Allegra' in site:
                TR = 1.6
            elif 'Cape-Town-UCT-Skyra' in site:
                TR = 1.73
            elif 'Chiba-CHB' in site:
                TR = 2.3
            elif 'Chiba-CHBC' in site:
                TR = 2.3 
            elif 'Chiba-CHBSRPB' in site:
                TR = 2.5 
            elif 'Dresden' in site:
                TR = 0.8 
            elif 'Kyoto-KPU-Kyoto1.5T' in site:
                TR = 2.411 
            elif 'Kyoto-KPU-Kyoto3T' in site:
                TR = 2
            elif 'Kyushu' in site:
                TR = 2.5
            elif 'Milan-HSR' in site:
                TR = 2
            elif 'New-York' in site:
                TR = 1
            elif 'NYSPI-Columbia-Adults' in site:
                TR = 0.85
            elif 'NYSPI-Columbia-Pediatric' in site:
                TR = 0.85
            elif 'Yale-Pittinger-HCP-Prisma' in site:
                TR = 0.8
            elif 'Yale-Pittinger-HCP-Trio' in site:
                TR = 0.7
            elif 'Yale-Pittinger-Yale-2014' in site:
                TR = 2
            elif 'Bangalore-NIMHANS' in site:
                TR = 2 
            elif 'Barcelone-Bellvitge-ANTIGA-1.5T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-COMPULSE-3T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-PROV-1.5T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-RESP-CBT-3T' in site:
                TR = 2
            elif 'Seoul-SNU' in site:
                TR = 3.5
            elif 'Shanghai-SMCH' in site:
                TR = 3
            elif 'UCLA' in site:
                TR = 2
            elif 'Vancouver-BCCHR' in site:
                TR = 2
            elif 'Yale-Gruner' in site:
                TR = 2
            else:
                raise ValueError(f"Site '{site}' does not have a defined TR value in TR_mappings. Please add it.")

            # Load the .npy file and calculate knee frequencies
            # sequence_length = 200
            y = np.load(npy_file_path).T
            sequence_length = y.shape[1]   # use the original number of points
            # y = y[20:20+sequence_length].T   
            # print(f"y: {y}")
            # print(f"y.shape: {y.shape}")

            try: 
                sample_whole = np.zeros(sequence_length,) # originally sequence_length   ## aggregates time-series data across ROIs   # sample_whole.shape = # of timepoints,

                ##### DEBUG STATEMENT #####
                # sample_whole = np.zeros(sequence_length - 20,)
                # print(f"sample_whole.shape: {sample_whole.shape}")
                ###########################

                intermediate_vec = y.shape[0]

                for i in range(intermediate_vec):
                    # print(f"y[i] shape: {y[i].shape}")
                    sample_whole+=y[i]

                sample_whole /= intermediate_vec    # averages the time-series signals (y) across a set number of ROIs

                T = TimeSeries(sample_whole, sampling_interval=TR)  # computes power spectral density (PSD) of the averaged time-series signal
                S_original = SpectralAnalyzer(T)

                # Lorentzian function fitting (dividing ultralow ~ low)  ## extracts the PSD data
                xdata = np.array(S_original.spectrum_fourier[0][1:])  # xdata = frequency values  
                ydata = np.abs(S_original.spectrum_fourier[1][1:])    # ydata = corresponding power values
                # print(f"xdata.shape: {xdata.shape}")
                # print(f"ydata.shape: {ydata.shape}")

                # initial parameter setting
                p0 = [0, 0.006]   
                param_bounds = ([-np.inf, 0], [np.inf, 1])

                # fitting Lorentzian function
                popt, pcov = curve_fit(lorentzian_function, xdata, ydata, p0=p0, maxfev = 5000, bounds=param_bounds)   # popt = optimal parameters

                f1 = popt[1]

                knee = round(popt[1]/(1/(sample_whole.shape[0]*TR)))   # calculates knee frequency 
                # print(f"knee: {knee}")

                if knee <= 0:
                    knee = 1

                if knee > ydata.shape[0]:
                    print(f"knee value: {knee}")
                    print(f"ydata.shape: {ydata.shape}")

                # divide low ~ high
                # initial parameter setting
                p1 = [2, 1, 23, 25, 0.16]
            
                # fitting multifractal function
                popt_mo, pcov = curve_fit(multi_fractal_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)   # fits a multi-fractal model to the high-frequency range (above the knee)
                pink = round(popt_mo[-1]/(1/(sample_whole.shape[0]*TR)))   # pink = an additional boundary
                f2 = popt_mo[-1]

                # Save values to the dictionaries
                # Check if the key exists in the dictionary
                if site in f1_dict:
                    # Append the value to the existing list
                    f1_dict[site].append(f1)
                    f2_dict[site].append(f2)
                    seq_len_dict[site].append(sequence_length)
                else:
                    # Create the key and initialize it with a list containing the value
                    f1_dict[site] = [f1]
                    f2_dict[site] = [f2]   
                    seq_len_dict[site] = [sequence_length] 
                print(f"Successfully processed {file}")  
            except Exception as e:
                print(f"Error processing: {file}")
                print(e)
                error_count += 1
                continue  # Skip the subject if an error occurs
            
print(f"Knee frequencies f1: {f1_dict}")
print(f"\nKnee frequencies f2: {f2_dict}")
print(f"Sequence lengths: {seq_len_dict}")
print(f"Error processing {error_count} files")


Error processing: Barcelona-HCPB_sub-008.npy
`ydata` must not be empty!
Successfully processed Brazil_sub-C002061.npy
Successfully processed Yale-Pittinger-HCP-Prisma_sub-YaleHCPPrismapb3225.npy
Successfully processed Seoul-SNU_sub-NOR117CSJ.npy
Successfully processed Dresden_sub-GEROME3073.npy
Successfully processed Bergen_sub-00059.npy
Successfully processed Bangalore-NIMHANS_sub-C0181.npy
Successfully processed Bangalore-NIMHANS_sub-C0128.npy
Error processing: Seoul-SNU_sub-DNO23LSM.npy
The number of func parameters=5 must not exceed the number of data points=3
Successfully processed Bangalore-NIMHANS_sub-ODP004.npy
Successfully processed Bangalore-NIMHANS_sub-ODP203.npy
knee value: 62
ydata.shape: (60,)
Error processing: Barcelone-Bellvitge-PROV-1.5T_sub-subIDIBELL15224.npy
`ydata` must not be empty!
Error processing: Barcelone-Bellvitge-ANTIGA-1.5T_sub-subIDIBELL15C032.npy
The number of func parameters=5 must not exceed the number of data points=2


/tmp/ipykernel_97204/4233512411.py:148: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_mo, pcov = curve_fit(multi_fractal_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)   # fits a multi-fractal model to the high-frequency range (above the knee)
/tmp/ipykernel_97204/1867528834.py:2: RuntimeWarning: overflow encountered in power
  return np.where(x < corner, A * x**beta_low, B * x**beta_high)
/tmp/ipykernel_97204/1867528834.py:2: RuntimeWarning: overflow encountered in multiply
  return np.where(x < corner, A * x**beta_low, B * x**beta_high)


Successfully processed Bangalore-NIMHANS_sub-ODP099.npy
Error processing: Chiba-CHB_sub-MADHC010.npy
The number of func parameters=5 must not exceed the number of data points=2
knee value: 253
ydata.shape: (149,)
Error processing: Yale-Pittinger-Yale-2014_sub-Yale2014AdultOCDtr7977.npy
`ydata` must not be empty!
Successfully processed Bergen_sub-00066.npy
Successfully processed Yale-Pittinger-Yale-2014_sub-Yale2014AdultOCDtb0314.npy
Successfully processed Barcelona-HCPB_sub-C0071.npy
Error processing: Barcelone-Bellvitge-ANTIGA-1.5T_sub-subIDIBELL15C042.npy
The number of func parameters=5 must not exceed the number of data points=3
Successfully processed Barcelona-HCPB_sub-042.npy
knee value: 65
ydata.shape: (60,)
Error processing: Barcelone-Bellvitge-ANTIGA-1.5T_sub-subIDIBELL15P67.npy
`ydata` must not be empty!
Successfully processed Braga-UMinho-Braga-3T_sub-MRI201905211BART003.npy
Successfully processed Barcelone-Bellvitge-RESP-CBT-3T_sub-RESPCBT25CTRPOST.npy
knee value: 82
ydata.s

### Check the results

In [57]:
# Summarize the data
data = []
for key in f1_dict.keys():
    f1_values = f1_dict[key]
    f2_values = f2_dict[key]
    seq_len_values = seq_len_dict[key]
    
    data.append({
        "key": key,
        "mean_f1": sum(f1_values) / len(f1_values) if f1_values else 0,
        "count_f1": len(f1_values),
        "mean_f2": sum(f2_values) / len(f2_values) if f2_values else 0,
        "count_f2": len(f2_values),
        "mean_seq_len": sum(seq_len_values) / len(seq_len_values) if seq_len_values else 0
    })

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)
print(f"\nTotal of {sum(df['count_f1'])} subjects")

                                key   mean_f1  count_f1  mean_f2  count_f2  \
0                            Brazil  0.163255        67     0.16        67   
1         Yale-Pittinger-HCP-Prisma  0.242802        60     0.16        60   
2                         Seoul-SNU  0.093670        59     0.16        59   
3                           Dresden  0.217902        35     0.16        35   
4                            Bergen  0.166548        45     0.16        45   
5                 Bangalore-NIMHANS  0.156027       277     0.16       277   
6          Yale-Pittinger-Yale-2014  0.146747        60     0.16        60   
7                    Barcelona-HCPB  0.154997        58     0.16        58   
8             Braga-UMinho-Braga-3T  0.256348        54     0.16        54   
9   Barcelone-Bellvitge-RESP-CBT-3T  0.156984        54     0.16        54   
10                    Chiba-CHBSRPB  0.137901        79     0.16        79   
11                      Yale-Gruner  0.156325        18     0.16

In [58]:
# Count the number of folders
folder_count = len([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))])

print(f"Number of folders in '{base_path}': {folder_count}")

Number of folders in '/pscratch/sd/p/pakmasha/MBBN_data': 2093


### Apply Gaussian kernel smoothing and calculate knee frequencies

In [ ]:
# Define the base path where the .tsv files are stored
base_path = '/pscratch/sd/p/pakmasha/MBBN_data'

# Create a dictionary to store knee values for each site
f1_dict = {}
f2_dict = {}
seq_len_dict = {}
error_count = 0

# Walk through all subdirectories and process .npy files
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.npy') and file.split('_')[-2] != "smoothed":
            # Construct full path to the .tsv file
            npy_file_path = os.path.join(root, file)
            # print(f"npy_file_path: {npy_file_path}")

            # Calculate the repetition time (TR) depending on the site
            site = file.split('_')[-2]
            if 'Amsterdam-AMC' in site:
                TR = 2.375
            elif 'Amsterdam-VUmc' in site:
                TR = 1.8
            elif 'Barcelona-HCPB' in site:
                TR = 2
            elif 'Bergen' in site:
                TR = 1.8
            elif 'Braga-UMinho-Braga-1.5T' in site:
                TR = 2
            elif 'Braga-UMinho-Braga-1.5T-act' in site:
                TR = 2
            elif 'Braga-UMinho-Braga-3T' in site:
                TR = 1
            elif 'Brazil' in site:
                TR = 2
            elif 'Cape-Town-UCT-Allegra' in site:
                TR = 1.6
            elif 'Cape-Town-UCT-Skyra' in site:
                TR = 1.73
            elif 'Chiba-CHB' in site:
                TR = 2.3
            elif 'Chiba-CHBC' in site:
                TR = 2.3 
            elif 'Chiba-CHBSRPB' in site:
                TR = 2.5 
            elif 'Dresden' in site:
                TR = 0.8 
            elif 'Kyoto-KPU-Kyoto1.5T' in site:
                TR = 2.411 
            elif 'Kyoto-KPU-Kyoto3T' in site:
                TR = 2
            elif 'Kyushu' in site:
                TR = 2.5
            elif 'Milan-HSR' in site:
                TR = 2
            elif 'New-York' in site:
                TR = 1
            elif 'NYSPI-Columbia-Adults' in site:
                TR = 0.85
            elif 'NYSPI-Columbia-Pediatric' in site:
                TR = 0.85
            elif 'Yale-Pittinger-HCP-Prisma' in site:
                TR = 0.8
            elif 'Yale-Pittinger-HCP-Trio' in site:
                TR = 0.7
            elif 'Yale-Pittinger-Yale-2014' in site:
                TR = 2
            elif 'Bangalore-NIMHANS' in site:
                TR = 2 
            elif 'Barcelone-Bellvitge-ANTIGA-1.5T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-COMPULSE-3T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-PROV-1.5T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-RESP-CBT-3T' in site:
                TR = 2
            elif 'Seoul-SNU' in site:
                TR = 3.5
            elif 'Shanghai-SMCH' in site:
                TR = 3
            elif 'UCLA' in site:
                TR = 2
            elif 'Vancouver-BCCHR' in site:
                TR = 2
            elif 'Yale-Gruner' in site:
                TR = 2
            else:
                raise ValueError(f"Site '{site}' does not have a defined TR value in TR_mappings. Please add it.")

            # Load the .npy file and calculate knee frequencies
            y = np.load(npy_file_path).T
            sequence_length = y.shape[1]   # use the original number of points
            # print(f"y: {y}")
            # print(f"y.shape: {y.shape}")

            try: 
                sample_whole = np.zeros(sequence_length,) # originally sequence_length   ## aggregates time-series data across ROIs   # sample_whole.shape = # of timepoints,

                ##### DEBUG STATEMENT #####
                # sample_whole = np.zeros(sequence_length - 20,)
                # print(f"sample_whole.shape: {sample_whole.shape}")
                ###########################

                intermediate_vec = y.shape[0]

                for i in range(intermediate_vec):
                    # print(f"y[i] shape: {y[i].shape}")
                    sample_whole+=y[i]

                sample_whole /= intermediate_vec    # averages the time-series signals (y) across a set number of ROIs

                # Smooth the averaged time series
                fwhm = 2
                smoothed_sample_whole = gaussian_smoothing_with_fwhm(sample_whole, fwhm)

                T = TimeSeries(smoothed_sample_whole, sampling_interval=TR)  # computes power spectral density (PSD) of the averaged time-series signal
                S_original = SpectralAnalyzer(T)

                # Lorentzian function fitting (dividing ultralow ~ low)  ## extracts the PSD data
                xdata = np.array(S_original.spectrum_fourier[0][1:])  # xdata = frequency values  
                ydata = np.abs(S_original.spectrum_fourier[1][1:])    # ydata = corresponding power values
                # print(f"xdata.shape: {xdata.shape}")
                # print(f"ydata.shape: {ydata.shape}")

                # initial parameter setting
                p0 = [0, 0.006]   
                param_bounds = ([-np.inf, 0], [np.inf, 1])

                # fitting Lorentzian function
                popt, pcov = curve_fit(lorentzian_function, xdata, ydata, p0=p0, maxfev = 5000, bounds=param_bounds)   # popt = optimal parameters

                f1 = popt[1]

                knee = round(popt[1]/(1/(smoothed_sample_whole.shape[0]*TR)))   # calculates knee frequency 
                # print(f"knee: {knee}")

                if knee <= 0:
                    knee = 1

                if knee > ydata.shape[0]:
                    print(f"knee value: {knee}")
                    print(f"ydata.shape: {ydata.shape}")

                # divide low ~ high
                # initial parameter setting
                p1 = [2, 1, 23, 25, 0.16]
            
                # fitting multifractal function
                popt_mo, pcov = curve_fit(multi_fractal_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)   # fits a multi-fractal model to the high-frequency range (above the knee)
                pink = round(popt_mo[-1]/(1/(smoothed_sample_whole.shape[0]*TR)))   # pink = an additional boundary
                f2 = popt_mo[-1]

                if file == "Brazil_sub-C001419_smoothed_2mm.npy":
                    print("file == 'Brazil_sub-C001419_smoothed_2mm.npy'")
                    print(f"y: {y[:2,]}")
                    print(f"smoothed_sample_whole: {smoothed_sample_whole[:2,]}")
                    print(f"knee: {knee}")
                    print(f"f1: {f1}, f2: {f2}")

                # Save values to the dictionaries
                # Check if the key exists in the dictionary
                if site in f1_dict:
                    # Append the value to the existing list
                    f1_dict[site].append(f1)
                    f2_dict[site].append(f2)
                    seq_len_dict[site].append(sequence_length)
                else:
                    # Create the key and initialize it with a list containing the value
                    f1_dict[site] = [f1]
                    f2_dict[site] = [f2]   
                    seq_len_dict[site] = [sequence_length] 
                print(f"Successfully processed {file}")  
            except Exception as e:
                print(f"Error processing: {file}")
                print(e)
                error_count += 1
                continue  # Skip the subject if an error occurs
            
print(f"Knee frequencies f1: {f1_dict}")
print(f"\nKnee frequencies f2: {f2_dict}")
print(f"Sequence lengths: {seq_len_dict}")
print(f"Error processing {error_count} files")


/tmp/ipykernel_97204/1867528834.py:2: RuntimeWarning: overflow encountered in power
  return np.where(x < corner, A * x**beta_low, B * x**beta_high)
/tmp/ipykernel_97204/1867528834.py:2: RuntimeWarning: overflow encountered in multiply
  return np.where(x < corner, A * x**beta_low, B * x**beta_high)
/tmp/ipykernel_97204/241008320.py:152: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_mo, pcov = curve_fit(multi_fractal_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)   # fits a multi-fractal model to the high-frequency range (above the knee)


Successfully processed Barcelona-HCPB_sub-008.npy
Successfully processed Brazil_sub-C002061.npy
Successfully processed Yale-Pittinger-HCP-Prisma_sub-YaleHCPPrismapb3225.npy
Successfully processed Seoul-SNU_sub-NOR117CSJ.npy
Successfully processed Dresden_sub-GEROME3073.npy
Successfully processed Bergen_sub-00059.npy
Successfully processed Bangalore-NIMHANS_sub-C0181.npy
Successfully processed Bangalore-NIMHANS_sub-C0128.npy
Successfully processed Seoul-SNU_sub-DNO23LSM.npy
Successfully processed Bangalore-NIMHANS_sub-ODP004.npy
Successfully processed Bangalore-NIMHANS_sub-ODP203.npy
Successfully processed Barcelone-Bellvitge-PROV-1.5T_sub-subIDIBELL15224.npy
Successfully processed Barcelone-Bellvitge-ANTIGA-1.5T_sub-subIDIBELL15C032.npy
Successfully processed Bangalore-NIMHANS_sub-ODP099.npy
Successfully processed Chiba-CHB_sub-MADHC010.npy
Successfully processed Yale-Pittinger-Yale-2014_sub-Yale2014AdultOCDtr7977.npy
Successfully processed Bergen_sub-00066.npy
Successfully processed Y

In [31]:
# Summarize the data
data = []
for key in f1_dict.keys():
    f1_values = f1_dict[key]
    f2_values = f2_dict[key]
    seq_len_values = seq_len_dict[key]
    
    data.append({
        "key": key,
        "mean_f1": sum(f1_values) / len(f1_values) if f1_values else 0,
        "count_f1": len(f1_values),
        "mean_f2": sum(f2_values) / len(f2_values) if f2_values else 0,
        "count_f2": len(f2_values),
        "mean_seq_len": sum(seq_len_values) / len(seq_len_values) if seq_len_values else 0
    })

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)
print(f"\nTotal of {sum(df['count_f1'])} subjects")

                                key   mean_f1  count_f1  mean_f2  count_f2  \
0                    Barcelona-HCPB  0.061647        64     0.16        64   
1                            Brazil  0.064681        93     0.16        93   
2         Yale-Pittinger-HCP-Prisma  0.094776        61     0.16        61   
3                         Seoul-SNU  0.039113        95     0.16        95   
4                           Dresden  0.091790        35     0.16        35   
5                            Bergen  0.066883        58     0.16        58   
6                 Bangalore-NIMHANS  0.064914       358     0.16       358   
7     Barcelone-Bellvitge-PROV-1.5T  0.066625        62     0.16        62   
8   Barcelone-Bellvitge-ANTIGA-1.5T  0.065806       131     0.16       131   
9                         Chiba-CHB  0.043987        44     0.16        44   
10         Yale-Pittinger-Yale-2014  0.058675        85     0.16        85   
11            Braga-UMinho-Braga-3T  0.093070        56     0.16

### Check the number of valid subjects depending on the sequence length

In [7]:
# Define the base path where the .tsv files are stored
base_path = '/pscratch/sd/p/pakmasha/MBBN_data'

# Create a dictionary to store knee values for each site
f1_dict = {}
f2_dict = {}
seq_len_dict = {}
error_count = 0

# Walk through all subdirectories and process .npy files
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('_smoothed_4mm.npy'):
            # Construct full path to the .tsv file
            npy_file_path = os.path.join(root, file)
            # print(f"npy_file_path: {npy_file_path}")

            # Calculate the repetition time (TR) depending on the site
            site = file.split('_')[-4]
            if 'Amsterdam-AMC' in site:
                TR = 2.375
            elif 'Amsterdam-VUmc' in site:
                TR = 1.8
            elif 'Barcelona-HCPB' in site:
                TR = 2
            elif 'Bergen' in site:
                TR = 1.8
            elif 'Braga-UMinho-Braga-1.5T' in site:
                TR = 2
            elif 'Braga-UMinho-Braga-1.5T-act' in site:
                TR = 2
            elif 'Braga-UMinho-Braga-3T' in site:
                TR = 1
            elif 'Brazil' in site:
                TR = 2
            elif 'Cape-Town-UCT-Allegra' in site:
                TR = 1.6
            elif 'Cape-Town-UCT-Skyra' in site:
                TR = 1.73
            elif 'Chiba-CHB' in site:
                TR = 2.3
            elif 'Chiba-CHBC' in site:
                TR = 2.3 
            elif 'Chiba-CHBSRPB' in site:
                TR = 2.5 
            elif 'Dresden' in site:
                TR = 0.8 
            elif 'Kyoto-KPU-Kyoto1.5T' in site:
                TR = 2.411 
            elif 'Kyoto-KPU-Kyoto3T' in site:
                TR = 2
            elif 'Kyushu' in site:
                TR = 2.5
            elif 'Milan-HSR' in site:
                TR = 2
            elif 'New-York' in site:
                TR = 1
            elif 'NYSPI-Columbia-Adults' in site:
                TR = 0.85
            elif 'NYSPI-Columbia-Pediatric' in site:
                TR = 0.85
            elif 'Yale-Pittinger-HCP-Prisma' in site:
                TR = 0.8
            elif 'Yale-Pittinger-HCP-Trio' in site:
                TR = 0.7
            elif 'Yale-Pittinger-Yale-2014' in site:
                TR = 2
            elif 'Bangalore-NIMHANS' in site:
                TR = 2 
            elif 'Barcelone-Bellvitge-ANTIGA-1.5T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-COMPULSE-3T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-PROV-1.5T' in site:
                TR = 2
            elif 'Barcelone-Bellvitge-RESP-CBT-3T' in site:
                TR = 2
            elif 'Seoul-SNU' in site:
                TR = 3.5
            elif 'Shanghai-SMCH' in site:
                TR = 3
            elif 'UCLA' in site:
                TR = 2
            elif 'Vancouver-BCCHR' in site:
                TR = 2
            elif 'Yale-Gruner' in site:
                TR = 2
            else:
                raise ValueError(f"Site '{site}' does not have a defined TR value in TR_mappings. Please add it.")

            # Load the .npy file and calculate knee frequencies
            sequence_length = 100
            y = np.load(npy_file_path)[20:20+sequence_length].T   
            # print(f"y: {y}")
            # print(f"y.shape: {y.shape}")

            try: 
                sample_whole = np.zeros(sequence_length,) # originally sequence_length   ## aggregates time-series data across ROIs   # sample_whole.shape = # of timepoints,

                ##### DEBUG STATEMENT #####
                # sample_whole = np.zeros(sequence_length - 20,)
                # print(f"sample_whole.shape: {sample_whole.shape}")
                ###########################

                intermediate_vec = y.shape[0]

                for i in range(intermediate_vec):
                    # print(f"y[i] shape: {y[i].shape}")
                    sample_whole+=y[i]

                sample_whole /= intermediate_vec    # averages the time-series signals (y) across a set number of ROIs

                # Smooth the averaged time series
                # fwhm = 2
                # smoothed_sample_whole = gaussian_smoothing_with_fwhm(sample_whole, fwhm)
                smoothed_sample_whole = sample_whole

                T = TimeSeries(smoothed_sample_whole, sampling_interval=TR)  # computes power spectral density (PSD) of the averaged time-series signal
                S_original = SpectralAnalyzer(T)

                # Lorentzian function fitting (dividing ultralow ~ low)  ## extracts the PSD data
                xdata = np.array(S_original.spectrum_fourier[0][1:])  # xdata = frequency values  
                ydata = np.abs(S_original.spectrum_fourier[1][1:])    # ydata = corresponding power values
                # print(f"xdata.shape: {xdata.shape}")
                # print(f"ydata.shape: {ydata.shape}")

                # initial parameter setting
                p0 = [0, 0.006]   
                param_bounds = ([-np.inf, 0], [np.inf, 1])

                # fitting Lorentzian function
                popt, pcov = curve_fit(lorentzian_function, xdata, ydata, p0=p0, maxfev = 5000, bounds=param_bounds)   # popt = optimal parameters

                f1 = popt[1]

                knee = round(popt[1]/(1/(smoothed_sample_whole.shape[0]*TR)))   # calculates knee frequency 
                # print(f"knee: {knee}")

                if knee <= 0:
                    knee = 1

                if knee > ydata.shape[0]:
                    print(f"knee value: {knee}")
                    print(f"ydata.shape: {ydata.shape}")

                # divide low ~ high
                # initial parameter setting
                p1 = [2, 1, 23, 25, 0.16]
            
                # fitting multifractal function
                popt_mo, pcov = curve_fit(multi_fractal_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)   # fits a multi-fractal model to the high-frequency range (above the knee)
                pink = round(popt_mo[-1]/(1/(smoothed_sample_whole.shape[0]*TR)))   # pink = an additional boundary
                f2 = popt_mo[-1]

                # Save values to the dictionaries
                # Check if the key exists in the dictionary
                if site in f1_dict:
                    # Append the value to the existing list
                    f1_dict[site].append(f1)
                    f2_dict[site].append(f2)
                    seq_len_dict[site].append(sequence_length)
                else:
                    # Create the key and initialize it with a list containing the value
                    f1_dict[site] = [f1]
                    f2_dict[site] = [f2]   
                    seq_len_dict[site] = [sequence_length] 
                print(f"Successfully processed {file}")  
            except Exception as e:
                print(f"Error processing: {file}")
                print(e)
                error_count += 1
                continue  # Skip the subject if an error occurs
            
print(f"Knee frequencies f1: {f1_dict}")
print(f"\nKnee frequencies f2: {f2_dict}")
print(f"Sequence lengths: {seq_len_dict}")
print(f"Error processing {error_count} files")


/tmp/ipykernel_808400/1867528834.py:2: RuntimeWarning: overflow encountered in power
  return np.where(x < corner, A * x**beta_low, B * x**beta_high)
/tmp/ipykernel_808400/1867528834.py:2: RuntimeWarning: overflow encountered in multiply
  return np.where(x < corner, A * x**beta_low, B * x**beta_high)
/tmp/ipykernel_808400/2934793644.py:151: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_mo, pcov = curve_fit(multi_fractal_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)   # fits a multi-fractal model to the high-frequency range (above the knee)


Successfully processed Barcelona-HCPB_sub-008_smoothed_4mm.npy
Successfully processed Brazil_sub-C002061_smoothed_4mm.npy
Successfully processed Yale-Pittinger-HCP-Prisma_sub-YaleHCPPrismapb3225_smoothed_4mm.npy
Error processing: Seoul-SNU_sub-NOR117CSJ_smoothed_4mm.npy
operands could not be broadcast together with shapes (100,) (92,) (100,) 
Successfully processed Dresden_sub-GEROME3073_smoothed_4mm.npy
Successfully processed Bergen_sub-00059_smoothed_4mm.npy
Successfully processed Bangalore-NIMHANS_sub-C0181_smoothed_4mm.npy
Successfully processed Bangalore-NIMHANS_sub-C0128_smoothed_4mm.npy
Error processing: Seoul-SNU_sub-DNO23LSM_smoothed_4mm.npy
operands could not be broadcast together with shapes (100,) (92,) (100,) 
Successfully processed Bangalore-NIMHANS_sub-ODP004_smoothed_4mm.npy
Successfully processed Bangalore-NIMHANS_sub-ODP203_smoothed_4mm.npy
Successfully processed Barcelone-Bellvitge-PROV-1.5T_sub-subIDIBELL15224_smoothed_4mm.npy
Successfully processed Barcelone-Bellvi

In [9]:
# Summarize the data
data = []
for key in f1_dict.keys():
    f1_values = f1_dict[key]
    f2_values = f2_dict[key]
    seq_len_values = seq_len_dict[key]
    
    data.append({
        "key": key,
        "mean_f1": sum(f1_values) / len(f1_values) if f1_values else 0,
        "count_f1": len(f1_values),
        "mean_f2": sum(f2_values) / len(f2_values) if f2_values else 0,
        "count_f2": len(f2_values),
        "mean_seq_len": sum(seq_len_values) / len(seq_len_values) if seq_len_values else 0
    })

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)
print(f"\nTotal of {sum(df['count_f1'])} subjects")

                                key   mean_f1  count_f1  mean_f2  count_f2  \
0                    Barcelona-HCPB  0.035169        64     0.16        64   
1                            Brazil  0.036814        93     0.16        93   
2         Yale-Pittinger-HCP-Prisma  0.061213        61     0.16        61   
3                           Dresden  0.059800        35     0.16        35   
4                            Bergen  0.035305        58     0.16        58   
5                 Bangalore-NIMHANS  0.037332       358     0.16       358   
6     Barcelone-Bellvitge-PROV-1.5T  0.037758        62     0.16        62   
7   Barcelone-Bellvitge-ANTIGA-1.5T  0.037059       131     0.16       131   
8                         Chiba-CHB  0.024788        44     0.16        44   
9          Yale-Pittinger-Yale-2014  0.035559        85     0.16        85   
10            Braga-UMinho-Braga-3T  0.064156        56     0.16        56   
11  Barcelone-Bellvitge-RESP-CBT-3T  0.029350        57     0.16